In [40]:
from ngsolve.webgui import Draw

import numpy as np
import ngsolve as ng
import dill
import pickle

motor_npz = np.load('../meshes/motor.npz', allow_pickle = True)

geoOCC = motor_npz['geoOCC'].tolist()
m = motor_npz['m']; m_new = m
j3 = motor_npz['j3']

geoOCCmesh = geoOCC.GenerateMesh()
mesh = ng.Mesh(geoOCCmesh)
# mesh.Refine()

# Draw(mesh)

In [41]:
import sys
sys.path.insert(0,'../../') # adds parent directory

import pde
order = 2

MESH = pde.mesh.netgen(mesh.ngmesh)

phi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'phi', order = x)
curlphi_Hcurl = lambda x : pde.hcurl.assemble(MESH, space = 'N0', matrix = 'curlphi', order = x)

D = lambda x : pde.int.assemble(MESH, order = x)

myM = lambda x: phi_Hcurl(x)[0] @ D(x) @ phi_Hcurl(x)[0].T + \
                phi_Hcurl(x)[1] @ D(x) @ phi_Hcurl(x)[1].T

myK = lambda x: curlphi_Hcurl(x) @ D(x) @ curlphi_Hcurl(x).T

R = pde.tools.tree_cotree_gauge(MESH)

In [227]:
HCurl = ng.HCurl(mesh, order = 2, nograds = True, dirichlet = 'stator_outer')

u,v = HCurl.TnT()

K = ng.BilinearForm(HCurl)
K += (ng.curl(u)*ng.curl(v)+1e-6*u*v)*ng.dx
K.Assemble()

f = ng.LinearForm(HCurl)
f += ng.CF(1)*ng.curl(v)*ng.dx
# f += ng.CF((1,1))*v*ng.dx
f.Assemble()

# rows,cols,vals = K.mat.COO()
# import scipy.sparse as sp
# ngK = sp.csr_matrix((vals,(rows,cols)))

# np.linalg.matrix_rank(ngK.A)
# print(ngK.shape)

x = ng.GridFunction(HCurl)
x.vec.data += K.mat.Inverse(HCurl.FreeDofs())*f.vec

# Draw(x)

In [220]:
from mst import *
g = Graph(MESH.np)

edges = MESH.EdgesToVertices[:,:2]

for i in range(edges.shape[0]): g.addEdge(edges[i,0],edges[i,1],i)

g.KruskalMST()

tree_indices = np.array(g.MST)[:,2]
cotree_indices = np.setdiff1d(np.arange(MESH.NoEdges),tree_indices)

10016

In [232]:
import pyngcore as pyng
pyBA = pyng.BitArray(HCurl.FreeDofs())
# pyBA.Set(HCurl.FreeDofs())

pyBA[0] = 0

for _,j in enumerate(cotree_indices): pyBA.Set(j)
print(HCurl.FreeDofs())

0: 11111111111111111111111111111111111111111111111111
50: 11111111111111111111111111111111111111111111111111
100: 11111111111111111111111111111111111111111111111111
150: 11111111111111111111111111111111111111111111111111
200: 11111111111111111111111111111111111111111111111111
250: 11111111111111111111111111111111111111111111111111
300: 11111111111111111111111111111111111111111111111111
350: 11111111111111111111111111111111111111111111111111
400: 11111110101111111111111111111111111111111111111111
450: 11111111111111111111111111111111111111111111111111
500: 11111111111111111111111111111111111111111111111111
550: 11111111111111111111111111111111111111111111111111
600: 11111111111111111111111111111111111111111111111111
650: 11111111111111111111111111111111111111111111111111
700: 11111111111111111111111111111111111111111111111111
750: 11111111111111111111111111111111111111111111111111
800: 11111111111111111111111111111111111111111111111111
850: 1111111111111111111111111111111111111111111111

23328